# **RANDOM MODEL**

**Initial setup**

In [ ]:
!git clone https://github.com/microsoft/recommenders.git

Cloning into 'recommenders'...
remote: Enumerating objects: 37228, done.
remote: Counting objects: 100% (760/760), done.
remote: Compressing objects: 100% (269/269), done.
remote: Total 37228 (delta 507), reused 637 (delta 475), pack-reused 36468
Receiving objects: 100% (37228/37228), 205.14 MiB | 21.25 MiB/s, done.
Resolving deltas: 100% (25151/25151), done.


In [ ]:
%cd recommenders

/content/recommenders


In [ ]:
!pip install retrying

In [ ]:
!pip install scrapbook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
#drive.mount('/content/drive')
import sys
import os
import numpy as np
import pandas as pd
import zipfile
from tqdm import tqdm
from tempfile import TemporaryDirectory
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.models.deeprec.deeprec_utils import download_deeprec_resources
from recommenders.models.newsrec.newsrec_utils import prepare_hparams
from recommenders.models.newsrec.models.nrms import NRMSModel
from recommenders.models.newsrec.io.mind_iterator import MINDIterator
from recommenders.models.newsrec.newsrec_utils import get_mind_data_set
from sklearn.metrics import ndcg_score
from recommenders.evaluation.python_evaluation import ndcg_at_k

import warnings
# Avoid printing some FutureWarnings
warnings.filterwarnings("ignore", category=FutureWarning)


print("System version: {}".format(sys.version))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.10.12 (main, Jun  7 2023, 12:45:35) [GCC 9.4.0]
Tensorflow version: 2.12.0


--------

In [ ]:
epochs = 5
seed = 42
batch_size = 32

# Options: demo, small, large
MIND_type = 'demo'

In [ ]:
tmpdir = TemporaryDirectory()
data_path = tmpdir.name

train_news_file = os.path.join(data_path, 'train', r'news.tsv')
train_behaviors_file = os.path.join(data_path, 'train', r'behaviors.tsv')
valid_news_file = os.path.join(data_path, 'valid', r'news.tsv')
valid_behaviors_file = os.path.join(data_path, 'valid', r'behaviors.tsv')
wordEmb_file = os.path.join(data_path, "utils", "embedding.npy")
userDict_file = os.path.join(data_path, "utils", "uid2index.pkl")
wordDict_file = os.path.join(data_path, "utils", "word_dict.pkl")
yaml_file = os.path.join(data_path, "utils", r'nrms.yaml')

mind_url, mind_train_dataset, mind_dev_dataset, mind_utils = get_mind_data_set(MIND_type)

if not os.path.exists(train_news_file):
    download_deeprec_resources(mind_url, os.path.join(data_path, 'train'), mind_train_dataset)

if not os.path.exists(valid_news_file):
    download_deeprec_resources(mind_url, \
                               os.path.join(data_path, 'valid'), mind_dev_dataset)
if not os.path.exists(yaml_file):
    download_deeprec_resources(r'https://recodatasets.z20.web.core.windows.net/newsrec/', \
                               os.path.join(data_path, 'utils'), mind_utils)

100%|██████████| 17.0k/17.0k [00:00<00:00, 34.1kKB/s]
100%|██████████| 9.84k/9.84k [00:00<00:00, 26.3kKB/s]
100%|██████████| 95.0k/95.0k [00:01<00:00, 77.7kKB/s]


In [ ]:
hparams = prepare_hparams(yaml_file,
                          wordEmb_file=wordEmb_file,
                          wordDict_file=wordDict_file,
                          userDict_file=userDict_file,
                          batch_size=batch_size,
                          epochs=epochs,
                          show_step=10)
print(hparams)

HParams object with values {'support_quick_scoring': True, 'dropout': 0.2, 'attention_hidden_dim': 200, 'head_num': 20, 'head_dim': 20, 'filter_num': 200, 'window_size': 3, 'vert_emb_dim': 100, 'subvert_emb_dim': 100, 'gru_unit': 400, 'type': 'ini', 'user_emb_dim': 50, 'learning_rate': 0.0001, 'optimizer': 'adam', 'epochs': 5, 'batch_size': 32, 'show_step': 10, 'title_size': 30, 'his_size': 50, 'data_format': 'news', 'npratio': 4, 'metrics': ['group_auc', 'mean_mrr', 'ndcg@5;10'], 'word_emb_dim': 300, 'model_type': 'nrms', 'loss': 'cross_entropy_loss', 'wordEmb_file': '/tmp/tmpv3xqojt1/utils/embedding.npy', 'wordDict_file': '/tmp/tmpv3xqojt1/utils/word_dict.pkl', 'userDict_file': '/tmp/tmpv3xqojt1/utils/uid2index.pkl'}


In [ ]:
yaml_file

'/tmp/tmpv3xqojt1/utils/nrms.yaml'

In [ ]:
iterator = MINDIterator

In [ ]:
model = NRMSModel(hparams, iterator, seed=seed)

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
%%time
model.fit(train_news_file, train_behaviors_file, valid_news_file, valid_behaviors_file)

step 1080 , total_loss: 1.5154, data_loss: 1.4191: : 1086it [02:30,  7.21it/s]
0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
586it [00:01, 473.65it/s]
236it [00:07, 29.85it/s]
7538it [00:00, 8522.51it/s]


at epoch 1
train info: logloss loss:1.5148877750663687
eval info: group_auc:0.58, mean_mrr:0.2449, ndcg@10:0.331, ndcg@5:0.2592
at epoch 1 , train time: 150.7 eval time: 19.8


step 1080 , total_loss: 1.4195, data_loss: 1.3300: : 1086it [02:26,  7.42it/s]
586it [00:00, 752.99it/s]
236it [00:07, 32.44it/s]
7538it [00:00, 8219.31it/s]


at epoch 2
train info: logloss loss:1.419488111692678
eval info: group_auc:0.6004, mean_mrr:0.2564, ndcg@10:0.3459, ndcg@5:0.2729
at epoch 2 , train time: 146.3 eval time: 18.7


step 1080 , total_loss: 1.3796, data_loss: 1.1564: : 1086it [02:26,  7.39it/s]
586it [00:00, 731.00it/s]
236it [00:07, 32.16it/s]
7538it [00:00, 8548.59it/s]


at epoch 3
train info: logloss loss:1.3796476611994468
eval info: group_auc:0.6048, mean_mrr:0.2637, ndcg@10:0.3543, ndcg@5:0.2829
at epoch 3 , train time: 146.9 eval time: 18.8


step 1080 , total_loss: 1.3509, data_loss: 1.2650: : 1086it [02:27,  7.38it/s]
586it [00:01, 515.49it/s]
236it [00:07, 32.45it/s]
7538it [00:00, 8481.42it/s]


at epoch 4
train info: logloss loss:1.3512131782526469
eval info: group_auc:0.6108, mean_mrr:0.2679, ndcg@10:0.3587, ndcg@5:0.2878
at epoch 4 , train time: 147.1 eval time: 18.8


step 1080 , total_loss: 1.3312, data_loss: 1.2909: : 1086it [02:27,  7.37it/s]
586it [00:01, 527.56it/s]
236it [00:07, 32.44it/s]
7538it [00:00, 8995.21it/s]


at epoch 5
train info: logloss loss:1.3312993496381995
eval info: group_auc:0.6139, mean_mrr:0.2691, ndcg@10:0.3621, ndcg@5:0.288
at epoch 5 , train time: 147.3 eval time: 18.7
CPU times: user 7min 25s, sys: 29.3 s, total: 7min 55s
Wall time: 13min 53s


In [ ]:
%%time
res_syn = model.run_eval(valid_news_file, valid_behaviors_file)
print(res_syn)

586it [00:00, 624.19it/s]
236it [00:07, 32.61it/s]
7538it [00:01, 5968.95it/s]


{'group_auc': 0.6139, 'mean_mrr': 0.2691, 'ndcg@5': 0.288, 'ndcg@10': 0.3621}
CPU times: user 18.2 s, sys: 1.26 s, total: 19.4 s
Wall time: 18.5 s


In [ ]:
#sb.glue("res_syn", res_syn)

In [ ]:
def convert_to_binary(probabilities, threshold):
    return [1 if prob >= threshold else 0 for prob in probabilities]

# Model Evaluation

Ground Truth

In [ ]:
# Read the file
behav_df_demo = pd.read_csv(valid_behaviors_file,sep='\t', header=None, names=['Impression_ID', 'User_ID', 'Time', 'History', 'Impressions'])
# Select a subset
behav_true_df = behav_df_demo.loc[:, ["Impression_ID", "User_ID", "Impressions"]]
# Create the Impressions_True
behav_true_df["Impressions_True"] = behav_true_df["Impressions"].str.split().apply(lambda x: [int(item.split("-")[1]) for item in x])
behav_true_df['Impressions_True_10'] = behav_true_df['Impressions_True'].apply(lambda x: x[:10])

#Display
behav_true_df.head()

,Impression_ID,User_ID,Impressions,Impressions_True,Impressions_True_10
0,1,U41827,N23699-0 N21291-0 N1901-0 N27292-0 N17443-0 N1...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1]"
1,2,U61881,N26916-0 N4641-0 N25522-0 N14893-0 N19035-0 N3...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,3,U54180,N13528-0 N27689-0 N10879-0 N11662-0 N14409-0 N...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,4,U41164,N20150-0 N1807-1 N26916-0 N28138-0 N9576-0 N19...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
4,5,U8588,N21325-0 N5982-0 N19737-1 N9576-0 N20150-0 N25...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"


Basic Model Predictions



In [ ]:
group_impr_indexes, group_labels, group_preds = model.run_fast_eval(valid_news_file, valid_behaviors_file)

586it [00:01, 498.30it/s]
236it [00:07, 32.66it/s]
7538it [00:00, 8245.07it/s]


In [ ]:
# Function to replace each element with its position in descending order
def replace_with_positions(array):
    order_array = np.argsort(-np.array(array))
    new_array = np.arange(1, len(array) + 1)[order_array]
    return new_array

In [ ]:
# Read the file
behav_df_model = pd.read_csv(valid_behaviors_file,sep='\t', header=None, names=['Impression_ID', 'User_ID', 'Time', 'History', 'Impressions'])
# Select a subset
behav_model_df = behav_df_model.loc[:, ["Impression_ID", "User_ID", "Impressions"]]
# Create a new column for the model predictions
model_pred = pd.Series([np.array(arr) for arr in group_preds])
behav_model_df["Model_Pred_Prob"] = model_pred
# Transform the model predictions into positional format
behav_model_df['Model_Pred'] = [replace_with_positions(array) for array in behav_model_df['Model_Pred_Prob']]
# Transform the model predictions into clicks o no clicks for accuracy
behav_model_df['Model_Pred_Clicks'] = behav_model_df['Model_Pred_Prob'].apply(lambda probabilities: convert_to_binary(probabilities, 0.9))
#Display
behav_model_df.head()

,Impression_ID,User_ID,Impressions,Model_Pred_Prob,Model_Pred,Model_Pred_Clicks
0,1,U41827,N23699-0 N21291-0 N1901-0 N27292-0 N17443-0 N1...,"[0.034063764, -0.10842401, -0.8481444, -0.2926...","[12, 9, 22, 21, 10, 23, 11, 27, 25, 1, 2, 17, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ..."
1,2,U61881,N26916-0 N4641-0 N25522-0 N14893-0 N19035-0 N3...,"[0.012356456, 0.8501639, 0.4330132, -0.4511937...","[25, 20, 41, 35, 52, 2, 26, 42, 28, 16, 32, 10...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,U54180,N13528-0 N27689-0 N10879-0 N11662-0 N14409-0 N...,"[0.32837206, 1.6309161, 0.32367486, -0.6508624...","[25, 2, 22, 5, 41, 44, 54, 20, 31, 33, 49, 12,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,U41164,N20150-0 N1807-1 N26916-0 N28138-0 N9576-0 N19...,"[-0.9434567, -0.7652998, 0.04328537, 0.6209965...","[10, 6, 4, 5, 11, 12, 3, 2, 9, 1, 7, 13, 8]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
4,5,U8588,N21325-0 N5982-0 N19737-1 N9576-0 N20150-0 N25...,"[0.69138265, 0.33198422, 1.4208741, 0.55464196...","[3, 10, 9, 1, 5, 4, 2, 6, 7, 8]","[0, 0, 1, 0, 0, 0, 0, 0, 1, 1]"


In [ ]:
behav_model_df.iloc[4,3]

array([ 0.69138265,  0.33198422,  1.4208741 ,  0.55464196,  0.62495995,
        0.30863857,  0.22213572, -0.6149236 ,  1.1555014 ,  1.2211335 ],
      dtype=float32)

In [ ]:
behav_model_df.iloc[4,4]

array([ 3, 10,  9,  1,  5,  4,  2,  6,  7,  8])

In [ ]:
def process_impression(impression_list):
    list_of_strings = impression_list.split()
    click = [x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '1']
    non_click = [x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '0']
    return click,non_click

# We can then indexize these two new columns:
behav_model_df['click'], behav_model_df['noclicks'] = zip(*behav_model_df['Impressions'].map(process_impression))

In [ ]:
behav_model_df["Clicks_count"] = behav_model_df["click"].apply(len)

In [ ]:
behav_model_df[["Model_Pred","Clicks_count"]]

,Model_Pred,Clicks_count
0,"[12, 9, 22, 21, 10, 23, 11, 27, 25, 1, 2, 17, ...",1
1,"[25, 20, 41, 35, 52, 2, 26, 42, 28, 16, 32, 10...",1
2,"[25, 2, 22, 5, 41, 44, 54, 20, 31, 33, 49, 12,...",1
3,"[10, 6, 4, 5, 11, 12, 3, 2, 9, 1, 7, 13, 8]",2
4,"[3, 10, 9, 1, 5, 4, 2, 6, 7, 8]",1
...,...,...
7533,"[5, 38, 31, 34, 44, 21, 42, 2, 41, 25, 18, 40,...",2
7534,"[8, 6, 2, 14, 17, 9, 7, 4, 12, 1, 13, 10, 5, 1...",1
7535,"[33, 25, 22, 9, 39, 38, 16, 10, 15, 7, 18, 19,...",1
7536,"[39, 21, 2, 41, 20, 56, 7, 55, 31, 66, 42, 49,...",1


In [ ]:
import numpy as np

# Your existing DataFrame
# Create the new array column
behav_model_df['New_Array'] = behav_model_df.apply(lambda row: [1] * row['Clicks_count'] + [0] * (len(row['Model_Pred']) - row['Clicks_count']), axis=1)
behav_model_df['Sorted_New_Array'] = behav_model_df.apply(lambda row: [x for _, x in sorted(zip(row['Model_Pred'], row['New_Array']))], axis=1)

# Print the updated DataFrame
behav_model_df

,Impression_ID,User_ID,Impressions,Model_Pred_Prob,Model_Pred,Model_Pred_Clicks,click,noclicks,Clicks_count,New_Array,Sorted_New_Array
0,1,U41827,N23699-0 N21291-0 N1901-0 N27292-0 N17443-0 N1...,"[0.034063764, -0.10842401, -0.8481444, -0.2926...","[12, 9, 22, 21, 10, 23, 11, 27, 25, 1, 2, 17, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...",[N8620],"[N23699, N21291, N1901, N27292, N17443, N18282...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
1,2,U61881,N26916-0 N4641-0 N25522-0 N14893-0 N19035-0 N3...,"[0.012356456, 0.8501639, 0.4330132, -0.4511937...","[25, 20, 41, 35, 52, 2, 26, 42, 28, 16, 32, 10...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[N19829],"[N26916, N4641, N25522, N14893, N19035, N3877,...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,U54180,N13528-0 N27689-0 N10879-0 N11662-0 N14409-0 N...,"[0.32837206, 1.6309161, 0.32367486, -0.6508624...","[25, 2, 22, 5, 41, 44, 54, 20, 31, 33, 49, 12,...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[N13530],"[N13528, N27689, N10879, N11662, N14409, N6849...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,U41164,N20150-0 N1807-1 N26916-0 N28138-0 N9576-0 N19...,"[-0.9434567, -0.7652998, 0.04328537, 0.6209965...","[10, 6, 4, 5, 11, 12, 3, 2, 9, 1, 7, 13, 8]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]","[N1807, N16798]","[N20150, N26916, N28138, N9576, N19737, N24553...",2,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]"
4,5,U8588,N21325-0 N5982-0 N19737-1 N9576-0 N20150-0 N25...,"[0.69138265, 0.33198422, 1.4208741, 0.55464196...","[3, 10, 9, 1, 5, 4, 2, 6, 7, 8]","[0, 0, 1, 0, 0, 0, 0, 0, 1, 1]",[N19737],"[N21325, N5982, N9576, N20150, N25701, N10908,...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...
7533,7534,U23841,N26256-0 N28117-0 N2718-0 N16798-0 N27689-0 N6...,"[0.13368455, 0.6859709, -1.105706, -0.615812, ...","[5, 38, 31, 34, 44, 21, 42, 2, 41, 25, 18, 40,...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[N227, N14073]","[N26256, N28117, N2718, N16798, N27689, N6280,...",2,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7534,7535,U28014,N26670-0 N12794-0 N3390-0 N17443-0 N27292-0 N2...,"[0.09569243, 0.92421454, -0.54583454, 0.214073...","[8, 6, 2, 14, 17, 9, 7, 4, 12, 1, 13, 10, 5, 1...","[0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",[N1901],"[N26670, N12794, N3390, N17443, N27292, N21852...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
7535,7536,U89684,N17443-0 N16798-0 N24553-0 N26096-0 N15927-0 N...,"[0.59571004, 0.56415355, -0.08778175, 0.500878...","[33, 25, 22, 9, 39, 38, 16, 10, 15, 7, 18, 19,...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",[N9576],"[N17443, N16798, N24553, N26096, N15927, N2625...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7536,7537,U92611,N14850-0 N26647-0 N272-0 N22751-0 N21398-0 N26...,"[-0.9115801, 0.39798364, -0.21221192, -0.97310...","[39, 21, 2, 41, 20, 56, 7, 55, 31, 66, 42, 49,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[N6782],"[N14850, N26647, N272, N22751, N21398, N26916,...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
model_pred_eval = behav_model_df["Sorted_New_Array"]

In [ ]:
import numpy as np

base_array = np.array([0.5, 1.4, -1.5, 0.1])

order_array = np.argsort(-base_array)

new_array = np.arange(1, len(base_array) + 1)[order_array]
new_array

array([2, 1, 4, 3])

In [ ]:
def process_impression(impression_list):
    list_of_strings = impression_list.split()
    click = [x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '1']
    non_click = [x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '0']
    return click,non_click

In [ ]:
def dcg_score(y_true, y_score, k=10):
    """Computing dcg score metric at k.

    Args:
        y_true (np.ndarray): Ground-truth labels.
        y_score (np.ndarray): Predicted labels.

    Returns:
        np.ndarray: dcg scores.
    """
    k = min(np.shape(y_true)[-1], k)
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)

In [ ]:
def ndcg_score(y_true, y_score, k=10):
    """Computing ndcg score metric at k.

    Args:
        y_true (np.ndarray): Ground-truth labels.
        y_score (np.ndarray): Predicted labels.

    Returns:
        numpy.ndarray: ndcg scores.
    """
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best

Random Model Predictions

In [ ]:
# Create the random probabilities for the predicitions
def random_model_prob(length):
    return np.random.rand(length)

In [ ]:
# Read the file
behav_df_demo = pd.read_csv(valid_behaviors_file,sep='\t', header=None, names=['Impression_ID', 'User_ID', 'Time', 'History', 'Impressions'])
# Select a subset
behav_random_df = behav_df_demo.loc[:, ["Impression_ID", "User_ID", "Impressions"]]
# Creation of Impressions_list
behav_random_df["Impressions_count"] = behav_random_df["Impressions"].str.split().apply(len)
# Creation of the Random Probabilities
behav_random_df['Random_Pred_Prob'] = behav_random_df['Impressions_count'].apply(lambda length: random_model_prob(length))
# Creation of the Random_Pred
#behav_random_df["Random_Pred"] = behav_random_df["Impressions_count"].apply(lambda x: np.random.permutation(np.arange(1, x+1)))
behav_random_df['Random_Pred'] = [replace_with_positions(x) for x in behav_random_df['Random_Pred_Prob']]
# Creation of the Random_Pred_Clicks
behav_random_df['Random_Pred_Clicks'] = behav_random_df['Random_Pred_Prob'].apply(lambda probabilities: convert_to_binary(probabilities, 0.9))
behav_random_df['Random_Pred_Clicks_10'] = behav_random_df['Random_Pred_Clicks'].apply(lambda x: x[:10])
# Display
behav_random_df.head()

,Impression_ID,User_ID,Impressions,Impressions_count,Random_Pred_Prob,Random_Pred,Random_Pred_Clicks,Random_Pred_Clicks_10
0,1,U41827,N23699-0 N21291-0 N1901-0 N27292-0 N17443-0 N1...,28,"[0.34052867486733485, 0.9011743188581268, 0.28...","[6, 21, 2, 15, 25, 9, 24, 26, 28, 13, 12, 7, 1...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0]"
1,2,U61881,N26916-0 N4641-0 N25522-0 N14893-0 N19035-0 N3...,61,"[0.09350352145013308, 0.11561108912473927, 0.4...","[61, 32, 34, 6, 12, 36, 47, 15, 37, 25, 58, 29...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0]"
2,3,U54180,N13528-0 N27689-0 N10879-0 N11662-0 N14409-0 N...,54,"[0.6918062021774947, 0.9189107762657492, 0.762...","[26, 40, 25, 41, 30, 33, 2, 14, 53, 47, 54, 16...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"
3,4,U41164,N20150-0 N1807-1 N26916-0 N28138-0 N9576-0 N19...,13,"[0.47570379828921594, 0.25565316124170323, 0.7...","[6, 8, 11, 3, 5, 13, 7, 9, 1, 4, 2, 12, 10]","[0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0]","[0, 0, 0, 0, 0, 1, 0, 1, 0, 0]"
4,5,U8588,N21325-0 N5982-0 N19737-1 N9576-0 N20150-0 N25...,10,"[0.5263562617142016, 0.8666203820450576, 0.387...","[4, 2, 6, 5, 10, 1, 3, 8, 9, 7]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
# We can then indexize these two new columns:
behav_random_df['click'], behav_random_df['noclicks'] = zip(*behav_random_df['Impressions'].map(process_impression))

In [ ]:
behav_random_df["Clicks_count"] = behav_random_df["click"].apply(len)

In [ ]:
behav_random_df[["Random_Pred","Clicks_count"]]

,Random_Pred,Clicks_count
0,"[21, 1, 15, 23, 26, 27, 6, 24, 9, 28, 2, 16, 1...",1
1,"[36, 26, 29, 60, 40, 22, 45, 34, 54, 42, 47, 2...",1
2,"[14, 10, 47, 11, 5, 44, 2, 43, 7, 19, 50, 6, 5...",1
3,"[11, 10, 2, 13, 7, 9, 3, 5, 8, 4, 6, 1, 12]",2
4,"[7, 6, 4, 5, 10, 8, 9, 2, 1, 3]",1
...,...,...
7533,"[10, 9, 8, 43, 47, 16, 28, 5, 23, 38, 3, 24, 4...",2
7534,"[11, 19, 5, 14, 6, 9, 16, 4, 7, 17, 8, 1, 2, 1...",1
7535,"[18, 13, 42, 5, 34, 9, 41, 40, 7, 15, 31, 33, ...",1
7536,"[50, 64, 20, 29, 58, 26, 46, 24, 45, 27, 3, 25...",1


In [ ]:
import numpy as np

# Your existing DataFrame
# Create the new array column
behav_random_df['New_Array'] = behav_random_df.apply(lambda row: [1] * row['Clicks_count'] + [0] * (len(row['Random_Pred']) - row['Clicks_count']), axis=1)
behav_random_df['Sorted_New_Array'] = behav_random_df.apply(lambda row: [x for _, x in sorted(zip(row['Random_Pred'], row['New_Array']))], axis=1)

# Print the updated DataFrame
behav_random_df

,Impression_ID,User_ID,Impressions,Impressions_count,Random_Pred_Prob,Random_Pred,Random_Pred_Clicks,click,noclicks,Clicks_count,New_Array,Sorted_New_Array
0,1,U41827,N23699-0 N21291-0 N1901-0 N27292-0 N17443-0 N1...,28,"[0.9218723954312412, 0.6327208359868325, 0.475...","[21, 1, 15, 23, 26, 27, 6, 24, 9, 28, 2, 16, 1...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[N8620],"[N23699, N21291, N1901, N27292, N17443, N18282...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,U61881,N26916-0 N4641-0 N25522-0 N14893-0 N19035-0 N3...,61,"[0.10155102077841816, 0.16019110970070216, 0.5...","[36, 26, 29, 60, 40, 22, 45, 34, 54, 42, 47, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[N19829],"[N26916, N4641, N25522, N14893, N19035, N3877,...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,U54180,N13528-0 N27689-0 N10879-0 N11662-0 N14409-0 N...,54,"[0.04852292332173058, 0.8822443241837339, 0.43...","[14, 10, 47, 11, 5, 44, 2, 43, 7, 19, 50, 6, 5...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, ...",[N13530],"[N13528, N27689, N10879, N11662, N14409, N6849...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,4,U41164,N20150-0 N1807-1 N26916-0 N28138-0 N9576-0 N19...,13,"[0.16955374998372008, 0.9685598261262819, 0.71...","[11, 10, 2, 13, 7, 9, 3, 5, 8, 4, 6, 1, 12]","[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1]","[N1807, N16798]","[N20150, N26916, N28138, N9576, N19737, N24553...",2,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0]"
4,5,U8588,N21325-0 N5982-0 N19737-1 N9576-0 N20150-0 N25...,10,"[0.2702893317939553, 0.35962229507493104, 0.06...","[7, 6, 4, 5, 10, 8, 9, 2, 1, 3]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]",[N19737],"[N21325, N5982, N9576, N20150, N25701, N10908,...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
7533,7534,U23841,N26256-0 N28117-0 N2718-0 N16798-0 N27689-0 N6...,49,"[0.21546196230150194, 0.006278800785487371, 0....","[10, 9, 8, 43, 47, 16, 28, 5, 23, 38, 3, 24, 4...","[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, ...","[N227, N14073]","[N26256, N28117, N2718, N16798, N27689, N6280,...",2,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ..."
7534,7535,U28014,N26670-0 N12794-0 N3390-0 N17443-0 N27292-0 N2...,19,"[0.32074490535279765, 0.3041069344727313, 0.24...","[11, 19, 5, 14, 6, 9, 16, 4, 7, 17, 8, 1, 2, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",[N1901],"[N26670, N12794, N3390, N17443, N27292, N21852...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
7535,7536,U89684,N17443-0 N16798-0 N24553-0 N26096-0 N15927-0 N...,43,"[0.5180564614094446, 0.30726354367102326, 0.14...","[18, 13, 42, 5, 34, 9, 41, 40, 7, 15, 31, 33, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",[N9576],"[N17443, N16798, N24553, N26096, N15927, N2625...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7536,7537,U92611,N14850-0 N26647-0 N272-0 N22751-0 N21398-0 N26...,67,"[0.21564004023794525, 0.2999249953817925, 0.74...","[50, 64, 20, 29, 58, 26, 46, 24, 45, 27, 3, 25...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[N6782],"[N14850, N26647, N272, N22751, N21398, N26916,...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
random_pred_eval = behav_random_df["Sorted_New_Array"]

In [ ]:
ndcg_list = [10]
for k in ndcg_list:
    ndcg_temp= np.mean(
        [
            ndcg_score(each_labels, each_preds, k)
            for each_labels, each_preds in zip(group_labels, group_preds)
        ]
    )

In [ ]:
ndcg_temp

0.3620933269616858

In [ ]:
ndcg_list = [10]
for k in ndcg_list:
    ndcg_temp= np.mean(
        [
            ndcg_score(each_labels, each_preds, k)
            for each_labels, each_preds in zip(group_labels, random_pred_eval)
        ]
    )

In [ ]:
ndcg_temp

0.2810605315385024

Popularity Model Predictions

In [ ]:
# Read the file
behav_df_demo = pd.read_csv(valid_behaviors_file,sep='\t', header=None, names=['Impression_ID', 'User_ID', 'Time', 'History', 'Impressions'])
# Select a subset
behav_pop_df = behav_df_demo.loc[:, ["Impression_ID", "User_ID", "Impressions"]]
# Split impressions
behav_pop_df["Code_Impressions_1"] = behav_pop_df["Impressions"].str.split().apply(lambda x: [item.split("-")[0] for item in x if item.split("-")[1] == "1"])
# Display
behav_pop_df

,Impression_ID,User_ID,Impressions,Code_Impressions_1
0,1,U41827,N23699-0 N21291-0 N1901-0 N27292-0 N17443-0 N1...,[N8620]
1,2,U61881,N26916-0 N4641-0 N25522-0 N14893-0 N19035-0 N3...,[N19829]
2,3,U54180,N13528-0 N27689-0 N10879-0 N11662-0 N14409-0 N...,[N13530]
3,4,U41164,N20150-0 N1807-1 N26916-0 N28138-0 N9576-0 N19...,"[N1807, N16798]"
4,5,U8588,N21325-0 N5982-0 N19737-1 N9576-0 N20150-0 N25...,[N19737]
...,...,...,...,...
7533,7534,U23841,N26256-0 N28117-0 N2718-0 N16798-0 N27689-0 N6...,"[N227, N14073]"
7534,7535,U28014,N26670-0 N12794-0 N3390-0 N17443-0 N27292-0 N2...,[N1901]
7535,7536,U89684,N17443-0 N16798-0 N24553-0 N26096-0 N15927-0 N...,[N9576]
7536,7537,U92611,N14850-0 N26647-0 N272-0 N22751-0 N21398-0 N26...,[N6782]


In [ ]:
# Create a new DataFrame to store unique keys and their counts
unique_keys_df = pd.DataFrame(columns=['news_code', 'count'])
# Iterate over each row in the original DataFrame
for index, row in behav_pop_df.iterrows():
  keys = row['Code_Impressions_1']
  # Iterate over each key in the list
  for key in keys:
      # Check if the key is already in the unique_keys_df
      if key in unique_keys_df['news_code'].values:
          # Increment the count for the existing key
          unique_keys_df.loc[unique_keys_df['news_code'] == key, 'count'] += 1
      else:
          # Add a new row with the key and count = 1
          unique_keys_df = unique_keys_df.append({'news_code': key, 'count': 1}, ignore_index=True)
# Sort it from popular to least popular
unique_keys_df = unique_keys_df.sort_values(by = "count", ascending = False).reset_index(drop = True)
# Display the result
unique_keys_df.head()

,news_code,count
0,N9576,818
1,N26508,476
2,N21325,404
3,N20150,388
4,N13530,291


In [ ]:
# Adding the column with the probabilities
unique_keys_df['pred_prob'] = 1  # Set all values to 1 initially
unique_keys_df.loc[unique_keys_df.index >= len(unique_keys_df) / 2, 'pred_prob'] = 0  # Set the second half to 0
unique_keys_df.head()

,news_code,count,pred_prob
0,N9576,818,1
1,N26508,476,1
2,N21325,404,1
3,N20150,388,1
4,N13530,291,1


In [ ]:
# Create an empty dictionary to store unique keys and their counts
unique_keys_dict = {}

# Iterate over each row in the original DataFrame
for index, row in behav_pop_df.iterrows():
    keys = row['Code_Impressions_1']
    # Iterate over each key in the list
    for key in keys:
        # Check if the key is already in the unique_keys_dict
        if key in unique_keys_dict:
            # Increment the count for the existing key
            unique_keys_dict[key] += 1
        else:
            # Add a new key with count = 1 to the dictionary
            unique_keys_dict[key] = 1

In [ ]:
# Dictionary with each news code and if it is going to be a click or not
popular_clicks_dict = dict(zip(unique_keys_df['news_code'], unique_keys_df['pred_prob']))

In [ ]:
# Define a function to generate the new array
def generate_new_array(arr):
    indexed_array = [(value, index) for index, value in enumerate(arr)]
    sorted_array = sorted(indexed_array, key=lambda x: x[0], reverse=True)
    new_array = [item[1] + 1 for item in sorted_array]
    return new_array

In [ ]:
# Function to transform the impresions into clicks or no clicks
def trans_imp_clicks_pop(arr, trans_dict):
    return [trans_dict[val] if val in trans_dict else 0 for val in arr]

In [ ]:
# Create a new column with the list of code
behav_pop_df["Codes_Impressions"] = behav_pop_df["Impressions"].str.split().apply(lambda x: [item.split("-")[0] for item in x])
# Creation of the Codes_Count
behav_pop_df['Codes_Count'] = behav_pop_df['Codes_Impressions'].map(lambda x: [unique_keys_dict.get(code, 0) for code in x])
# Creation of Popular_Pred
behav_pop_df['Popular_Pred'] = behav_pop_df['Codes_Count'].apply(generate_new_array)
# Creation of Popular_Pred_Clicks
behav_pop_df['Popular_Pred_Clicks'] = behav_pop_df['Codes_Impressions'].apply(lambda arr: trans_imp_clicks_pop(arr, popular_clicks_dict))
behav_pop_df['Popular_Pred_Clicks_10'] = behav_pop_df['Popular_Pred_Clicks'].apply(lambda x: x[:10])
# Display
behav_pop_df

,Impression_ID,User_ID,Impressions,Code_Impressions_1,Codes_Impressions,Codes_Count,Popular_Pred,Popular_Pred_Clicks,Popular_Pred_Clicks_10
0,1,U41827,N23699-0 N21291-0 N1901-0 N27292-0 N17443-0 N1...,[N8620],"[N23699, N21291, N1901, N27292, N17443, N18282...","[222, 2, 72, 16, 111, 99, 58, 52, 86, 9, 157, ...","[1, 21, 12, 15, 11, 20, 19, 5, 6, 28, 18, 9, 3...","[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 0, 1, 1, 1, 1, 1, 1, 1, 1]"
1,2,U61881,N26916-0 N4641-0 N25522-0 N14893-0 N19035-0 N3...,[N19829],"[N26916, N4641, N25522, N14893, N19035, N3877,...","[140, 67, 17, 20, 11, 26, 3, 64, 404, 282, 1, ...","[58, 20, 9, 10, 41, 47, 13, 16, 50, 1, 51, 59,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
2,3,U54180,N13528-0 N27689-0 N10879-0 N11662-0 N14409-0 N...,[N13530],"[N13528, N27689, N10879, N11662, N14409, N6849...","[12, 77, 18, 42, 94, 58, 31, 51, 111, 17, 0, 7...","[20, 49, 36, 42, 25, 22, 9, 5, 2, 6, 21, 33, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
3,4,U41164,N20150-0 N1807-1 N26916-0 N28138-0 N9576-0 N19...,"[N1807, N16798]","[N20150, N1807, N26916, N28138, N9576, N19737,...","[388, 166, 140, 157, 818, 282, 36, 28, 404, 21...","[5, 12, 9, 1, 6, 11, 10, 2, 4, 3, 7, 8, 13]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
4,5,U8588,N21325-0 N5982-0 N19737-1 N9576-0 N20150-0 N25...,[N19737],"[N21325, N5982, N19737, N9576, N20150, N25701,...","[404, 18, 282, 818, 388, 58, 27, 42, 476, 291]","[4, 9, 1, 5, 10, 3, 6, 8, 7, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
...,...,...,...,...,...,...,...,...,...
7533,7534,U23841,N26256-0 N28117-0 N2718-0 N16798-0 N27689-0 N6...,"[N227, N14073]","[N26256, N28117, N2718, N16798, N27689, N6280,...","[7, 33, 23, 27, 77, 3, 11, 58, 11, 6, 166, 21,...","[18, 27, 17, 14, 44, 24, 49, 11, 13, 21, 5, 8,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
7534,7535,U28014,N26670-0 N12794-0 N3390-0 N17443-0 N27292-0 N2...,[N1901],"[N26670, N12794, N3390, N17443, N27292, N21852...","[46, 86, 17, 111, 16, 214, 140, 197, 818, 72, ...","[9, 14, 6, 8, 18, 11, 7, 4, 17, 16, 2, 10, 12,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
7535,7536,U89684,N17443-0 N16798-0 N24553-0 N26096-0 N15927-0 N...,[N9576],"[N17443, N16798, N24553, N26096, N15927, N2625...","[111, 27, 36, 7, 16, 7, 55, 10, 282, 94, 7, 1,...","[34, 22, 31, 32, 9, 25, 15, 41, 38, 21, 1, 10,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
7536,7537,U92611,N14850-0 N26647-0 N272-0 N22751-0 N21398-0 N26...,[N6782],"[N14850, N26647, N272, N22751, N21398, N26916,...","[6, 11, 6, 0, 31, 140, 90, 170, 44, 10, 15, 79...","[66, 38, 17, 23, 21, 8, 19, 31, 65, 6, 39, 32,...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1]"


In [ ]:
def process_impression(impression_list):
    list_of_strings = impression_list.split()
    click = [x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '1']
    non_click = [x.split('-')[0] for x in list_of_strings if x.split('-')[1] == '0']
    return click,non_click

# We can then indexize these two new columns:
behav_pop_df['click'], behav_pop_df['noclicks'] = zip(*behav_pop_df['Impressions'].map(process_impression))

In [ ]:
behav_pop_df["Clicks_count"] = behav_pop_df["click"].apply(len)

In [ ]:
behav_pop_df

,Impression_ID,User_ID,Impressions,Code_Impressions_1,Codes_Impressions,Codes_Count,Popular_Pred,Popular_Pred_Clicks,click,noclicks,Clicks_count
0,1,U41827,N23699-0 N21291-0 N1901-0 N27292-0 N17443-0 N1...,[N8620],"[N23699, N21291, N1901, N27292, N17443, N18282...","[222, 2, 72, 16, 111, 99, 58, 52, 86, 9, 157, ...","[1, 21, 12, 15, 11, 20, 19, 5, 6, 28, 18, 9, 3...","[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[N8620],"[N23699, N21291, N1901, N27292, N17443, N18282...",1
1,2,U61881,N26916-0 N4641-0 N25522-0 N14893-0 N19035-0 N3...,[N19829],"[N26916, N4641, N25522, N14893, N19035, N3877,...","[140, 67, 17, 20, 11, 26, 3, 64, 404, 282, 1, ...","[58, 20, 9, 10, 41, 47, 13, 16, 50, 1, 51, 59,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, ...",[N19829],"[N26916, N4641, N25522, N14893, N19035, N3877,...",1
2,3,U54180,N13528-0 N27689-0 N10879-0 N11662-0 N14409-0 N...,[N13530],"[N13528, N27689, N10879, N11662, N14409, N6849...","[12, 77, 18, 42, 94, 58, 31, 51, 111, 17, 0, 7...","[20, 49, 36, 42, 25, 22, 9, 5, 2, 6, 21, 33, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...",[N13530],"[N13528, N27689, N10879, N11662, N14409, N6849...",1
3,4,U41164,N20150-0 N1807-1 N26916-0 N28138-0 N9576-0 N19...,"[N1807, N16798]","[N20150, N1807, N26916, N28138, N9576, N19737,...","[388, 166, 140, 157, 818, 282, 36, 28, 404, 21...","[5, 12, 9, 1, 6, 11, 10, 2, 4, 3, 7, 8, 13]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[N1807, N16798]","[N20150, N26916, N28138, N9576, N19737, N24553...",2
4,5,U8588,N21325-0 N5982-0 N19737-1 N9576-0 N20150-0 N25...,[N19737],"[N21325, N5982, N19737, N9576, N20150, N25701,...","[404, 18, 282, 818, 388, 58, 27, 42, 476, 291]","[4, 9, 1, 5, 10, 3, 6, 8, 7, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",[N19737],"[N21325, N5982, N9576, N20150, N25701, N10908,...",1
...,...,...,...,...,...,...,...,...,...,...,...
7533,7534,U23841,N26256-0 N28117-0 N2718-0 N16798-0 N27689-0 N6...,"[N227, N14073]","[N26256, N28117, N2718, N16798, N27689, N6280,...","[7, 33, 23, 27, 77, 3, 11, 58, 11, 6, 166, 21,...","[18, 27, 17, 14, 44, 24, 49, 11, 13, 21, 5, 8,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[N227, N14073]","[N26256, N28117, N2718, N16798, N27689, N6280,...",2
7534,7535,U28014,N26670-0 N12794-0 N3390-0 N17443-0 N27292-0 N2...,[N1901],"[N26670, N12794, N3390, N17443, N27292, N21852...","[46, 86, 17, 111, 16, 214, 140, 197, 818, 72, ...","[9, 14, 6, 8, 18, 11, 7, 4, 17, 16, 2, 10, 12,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[N1901],"[N26670, N12794, N3390, N17443, N27292, N21852...",1
7535,7536,U89684,N17443-0 N16798-0 N24553-0 N26096-0 N15927-0 N...,[N9576],"[N17443, N16798, N24553, N26096, N15927, N2625...","[111, 27, 36, 7, 16, 7, 55, 10, 282, 94, 7, 1,...","[34, 22, 31, 32, 9, 25, 15, 41, 38, 21, 1, 10,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...",[N9576],"[N17443, N16798, N24553, N26096, N15927, N2625...",1
7536,7537,U92611,N14850-0 N26647-0 N272-0 N22751-0 N21398-0 N26...,[N6782],"[N14850, N26647, N272, N22751, N21398, N26916,...","[6, 11, 6, 0, 31, 140, 90, 170, 44, 10, 15, 79...","[66, 38, 17, 23, 21, 8, 19, 31, 65, 6, 39, 32,...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...",[N6782],"[N14850, N26647, N272, N22751, N21398, N26916,...",1


In [ ]:
import numpy as np

# Your existing DataFrame
# Create the new array column
behav_pop_df['New_Array'] = behav_pop_df.apply(lambda row: [1] * row['Clicks_count'] + [0] * (len(row['Popular_Pred']) - row['Clicks_count']), axis=1)
behav_pop_df['Sorted_New_Array'] = behav_pop_df.apply(lambda row: [x for _, x in sorted(zip(row['Popular_Pred'], row['New_Array']))], axis=1)

# Print the updated DataFrame
behav_pop_df

,Impression_ID,User_ID,Impressions,Code_Impressions_1,Codes_Impressions,Codes_Count,Popular_Pred,Popular_Pred_Clicks,click,noclicks,Clicks_count,New_Array,Sorted_New_Array
0,1,U41827,N23699-0 N21291-0 N1901-0 N27292-0 N17443-0 N1...,[N8620],"[N23699, N21291, N1901, N27292, N17443, N18282...","[222, 2, 72, 16, 111, 99, 58, 52, 86, 9, 157, ...","[1, 21, 12, 15, 11, 20, 19, 5, 6, 28, 18, 9, 3...","[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[N8620],"[N23699, N21291, N1901, N27292, N17443, N18282...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,2,U61881,N26916-0 N4641-0 N25522-0 N14893-0 N19035-0 N3...,[N19829],"[N26916, N4641, N25522, N14893, N19035, N3877,...","[140, 67, 17, 20, 11, 26, 3, 64, 404, 282, 1, ...","[58, 20, 9, 10, 41, 47, 13, 16, 50, 1, 51, 59,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, ...",[N19829],"[N26916, N4641, N25522, N14893, N19035, N3877,...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,U54180,N13528-0 N27689-0 N10879-0 N11662-0 N14409-0 N...,[N13530],"[N13528, N27689, N10879, N11662, N14409, N6849...","[12, 77, 18, 42, 94, 58, 31, 51, 111, 17, 0, 7...","[20, 49, 36, 42, 25, 22, 9, 5, 2, 6, 21, 33, 4...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...",[N13530],"[N13528, N27689, N10879, N11662, N14409, N6849...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,4,U41164,N20150-0 N1807-1 N26916-0 N28138-0 N9576-0 N19...,"[N1807, N16798]","[N20150, N1807, N26916, N28138, N9576, N19737,...","[388, 166, 140, 157, 818, 282, 36, 28, 404, 21...","[5, 12, 9, 1, 6, 11, 10, 2, 4, 3, 7, 8, 13]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[N1807, N16798]","[N20150, N26916, N28138, N9576, N19737, N24553...",2,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]"
4,5,U8588,N21325-0 N5982-0 N19737-1 N9576-0 N20150-0 N25...,[N19737],"[N21325, N5982, N19737, N9576, N20150, N25701,...","[404, 18, 282, 818, 388, 58, 27, 42, 476, 291]","[4, 9, 1, 5, 10, 3, 6, 8, 7, 2]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",[N19737],"[N21325, N5982, N9576, N20150, N25701, N10908,...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7533,7534,U23841,N26256-0 N28117-0 N2718-0 N16798-0 N27689-0 N6...,"[N227, N14073]","[N26256, N28117, N2718, N16798, N27689, N6280,...","[7, 33, 23, 27, 77, 3, 11, 58, 11, 6, 166, 21,...","[18, 27, 17, 14, 44, 24, 49, 11, 13, 21, 5, 8,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[N227, N14073]","[N26256, N28117, N2718, N16798, N27689, N6280,...",2,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7534,7535,U28014,N26670-0 N12794-0 N3390-0 N17443-0 N27292-0 N2...,[N1901],"[N26670, N12794, N3390, N17443, N27292, N21852...","[46, 86, 17, 111, 16, 214, 140, 197, 818, 72, ...","[9, 14, 6, 8, 18, 11, 7, 4, 17, 16, 2, 10, 12,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",[N1901],"[N26670, N12794, N3390, N17443, N27292, N21852...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
7535,7536,U89684,N17443-0 N16798-0 N24553-0 N26096-0 N15927-0 N...,[N9576],"[N17443, N16798, N24553, N26096, N15927, N2625...","[111, 27, 36, 7, 16, 7, 55, 10, 282, 94, 7, 1,...","[34, 22, 31, 32, 9, 25, 15, 41, 38, 21, 1, 10,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, ...",[N9576],"[N17443, N16798, N24553, N26096, N15927, N2625...",1,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7536,7537,U92611,N14850-0 N26647-0 N272-0 N22751-0 N21398-0 N26...,[N6782],"[N14850, N26647, N272, N22751, N21398, N26916,...","[6, 11, 6, 0, 31, 140, 90, 170, 44, 10, 15, 79...","[66, 38, 17, 23, 21, 8, 19, 31, 65, 6, 39, 32,...","[1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, ...

In [ ]:
pop_pred_eval = behav_pop_df["Sorted_New_Array"]

In [ ]:
ndcg_list = [10]
for k in ndcg_list:
    ndcg_temp= np.mean(
        [
            ndcg_score(each_labels, each_preds, k)
            for each_labels, each_preds in zip(group_labels, pop_pred_eval)
        ]
    )

In [ ]:
ndcg_temp

0.3332973648492855

# MAP

In [ ]:
def score_map(true_labels, pred_labels):

  # AP array
  ap_array = []
  for y_true, y_pred in zip(true_labels, pred_labels):
    # Initialize ap
    ap = 0
    # Loop on the pred
    for index, value in enumerate(y_pred):
      # Check if it is a one
      if value == 1:
        # Add the app
        position = index + 1
        ap += 1 / position

    # Calculate the ap for the user
    ap = ap / sum(y_true)
    # Save it in the array of aps
    ap_array.append(ap)

  return sum(ap_array) / len(ap_array)


In [ ]:
score_map(group_labels, model_pred_eval)

0.21871805131777394

In [ ]:
score_map(group_labels, random_pred_eval)

0.2126092901851324

In [ ]:
score_map(group_labels, pop_pred_eval)

0.22081413258335672

In [ ]:
score_map([[0,1,0,0,1], [0,0,1,0,0]], [[0,0,1,0,0], [0,1,0,0,0]])

0.3333333333333333

## Precision

In [ ]:
from sklearn.metrics import accuracy_score, precision_score

# Convert the arrays of sequences to 1D arrays
true_labels = np.concatenate(behav_true_df["Impressions_True_10"].values)
#predicted_labels_model = np.concatenate(behav_model_df['Model_Pred_Clicks'].values)
predicted_labels_random = np.concatenate(behav_random_df['Random_Pred_Clicks_10'].values)
predicted_labels_popular = np.concatenate(behav_pop_df['Popular_Pred_Clicks_10'].values)

# Calculate overall accuracy
#overall_precision_model = precision_score(true_labels, predicted_labels_model)
overall_precision_random = precision_score(true_labels, predicted_labels_random)
overall_precision_popular = precision_score(true_labels, predicted_labels_popular)

#print("Overall Precision Model:", overall_precision_model)
print("Overall Precision Random:", overall_precision_random)
print("Overall Precision Popular:", overall_precision_popular)

Overall Precision Random: 0.06999560697027384
Overall Precision Popular: 0.07641879298155409


## PRECISION NOCHE


In [ ]:
import pickle as pk

gt_path = "/content/drive/MyDrive/Colab Notebooks/IMPACT_PROJECT/group_labels_night.pkl"
pred_path = "/content/drive/MyDrive/Colab Notebooks/IMPACT_PROJECT/group_preds_night.pkl"


with open(gt_path, "rb") as file:
    gt_data = pk.load(file)

In [ ]:
with open(pred_path, "rb") as file:
    pred_data = pk.load(file)

In [ ]:
def transform_to_binary_with_threshold(arr, threshold):
    return [1 if value >= threshold else 0 for value in arr]

In [ ]:
binary_result = transform_to_binary_with_threshold(np.concatenate(pred_data), 0.9)
print(binary_result)  # Output: [0, 1, 1, 0, 1]

[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
from sklearn.metrics import accuracy_score, precision_score

prec = precision_score(np.concatenate(gt_data), binary_result)

print("Precision: ", prec)

Precision:  0.11698704378016767
